# LEGO BLOCK CODE
Blocks of code that can be used to build the full bot.

## Required Libraries

In [1]:
# '''
#     ***************************************************
#     ***************************************************
#     Script: Vaccine Status Bot
#     Author: OZeidi, ozeidi91@gmail.com, Fatma Bait Ishaq
#     Description: Fetch Vacine Status from MOH COVID19 Website
#     Version: V1.0
#     Last Modification: 29th Sep 2021
#     ***************************************************
#     ***************************************************

from pandas.core.dtypes.missing import isna
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
####
# Needed for the wait_for_xpath fucntion
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec


import os
# from urllib.request import urlopen
import pandas as pd
import numpy as np
# import requests
# import csv
# import datetime
import time
import random
import pickle

## Browser Check code

In [ ]:
# '''
#     ***************************************************
#     ***************************************************
#     Script: Vaccine Status Bot
#     Author: OZeidi, ozeidi91@gmail.com, Fatma Bait Ishaq
#     Description: Fetch Vacine Status from MOH COVID19 Website
#     Version: V1.0
#     Last Modification: 29th Sep 2021
#     ***************************************************
#     ***************************************************

# Change this to the path of your chrome Driver
dirpath = "C:/Users/MU63344/OneDrive - Petroleum Development Oman/Desktop/New folder/chromedriver.exe"


def wait_for_xpath( broswer, xpath):
    wait = WebDriverWait(broswer, 10)
    ui = wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    ActionChains(broswer).move_to_element(ui).perform()
    return ui


def browser_options():
    options = Options()
    #options.add_argument("--start-maximized")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("user-agent=Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393")
    #options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #options.add_argument('--disable-gpu')
    #options.add_argument('disable-infobars')
    options.add_argument("--disable-extensions")
    return options


def start_browsing():
    res = {}
    
    browser = webdriver.Chrome(chrome_options=browser_options(), executable_path = dirpath )
    print("Loading the page\n ")
    browser.get("https://covid19.moh.gov.om/#/check-certificate")
    df = pd.read_excel('Data.xlsx')

    res = {}
    #for index, row in df.iloc[:10].iterrows():
    for index, row in df.iterrows():
        # print(row['ID No.'])
        if not (np.isnan(row['ID No.'])):
            mu = row['Comp. No']
            id = int(row['ID No.'])
            print(index)
            print(id)
            # we will insert the function to fetch vac details here
            vac= check_status(browser, id)
            # a dcitionary to hold the reults
            res[mu] = vac
            with open('Vac.pickle', 'wb') as f:
                pickle.dump(res, f)
            #logging.info(f'MU: {mu}, ID: {id}, DATA: {vac}')
            # Save the ditionary to Excel file
            df = pd.DataFrame(res, index=[0]).T.reset_index()
            df
            df.columns = ['MU', 'Vaccine']

            df.to_excel('results.xlsx')
        else:
            print('No ID')


    #print(res)


def check_status(browser, ID):
    try:
        #browser.get("https://covid19.moh.gov.om/#/check-certificate")
        
        civil_id_txtbox =  wait_for_xpath(browser,'//*[@id="mat-input-1"]')#browser.find_element_by_xpath('//*[@id="mat-input-1"]')
        civil_id_txtbox.send_keys(ID)
        # Click the button to check the status
        time.sleep(1)
        btn = wait_for_xpath(browser, '//*[@id="loader"]/main/vacccert/div/div[3]/div/button')
        btn.click()
        try:
            vac_table = wait_for_xpath(browser,'//*[@id="loader"]/main/vacccert/div/div[4]/div[2]/table')
            # vac_table = browser.find_element_by_xpath('//*[@id="loader"]/main/vacccert/div/div[4]/div[2]/table')
            vac_status = vac_table.text 
            # btn = wait_for_xpath(browser,'//*[@id="loader"]/main/vacccert/div/div[4]/div[3]/button')
            # btn.click()
            time.sleep(1)
            browser.execute_script("document.querySelector('#loader > main > vacccert > div > div.wizard-panel.ng-star-inserted > div.text-center > button').click()")
            
        except :
            vac_status = "NOT VACCINATED"
            browser.execute_script("document.querySelector('#loader > main > vacccert > div > div.wizard-panel.ng-star-inserted > div:nth-child(2) > button').click()")

        #browser.get("https://covid19.moh.gov.om/#/check-certificate")
        # time.sleep(1)
        #browser.execute_script("document.querySelector('#loader > main > vacccert > div > div.wizard-panel.ng-star-inserted > div.text-center > button').click()")

        # # WebDriverWait(browser,10).until(ec.element_to_be_clickable((By.XPATH,'//*[@id="loader"]/main/vacccert/div/div[4]/div[3]/button'))).click()
        # print('Return Back Clicked')
        return vac_status
    except TimeoutException:
        print("Error")
        browser.get("https://covid19.moh.gov.om/#/check-certificate")


start_browsing()



# Breakdown the Results into the final Excel File

In [8]:
df = pd.read_excel(open('results.xlsx', 'rb'), sheet_name='Sheet1')  
#df.apply (lambda row: (row['STATUS']).replace('\t', ''), axis=1)

#df['STATUS'].replace('\t', '').split('\n')
def break_list(lst):
    size = len(lst)
    idx_list = [idx + 1 for idx, val in
                enumerate(lst) if val == '']
    
    
    res = [lst[i: j] for i, j in
            zip([0] + idx_list, idx_list + 
            ([size] if idx_list[-1] != size else []))]
    return (res)


def parse(row):
    res = []
    try:
        txtbd = (row['Vaccine'].replace('\t', '').split('\n'))
        cnt =len(txtbd)

    except:
        cnt = 0

    if (cnt == 10):
        # print(txtbd)
        # txtbd = break_list(txtbd)
        
        res.append({'MU': row['MU'],

        'Dose': 1 , 
        'Vaccine':txtbd[1] , 
        'Location':txtbd[2] , 
        'Date': txtbd[3]})
        
        res.append({'MU': row['MU'],
        'Dose': 2 , 
        'Vaccine':txtbd[6], 
        'Location':txtbd[7] , 
        'Date': txtbd[8]})

    elif( cnt == 5 ):
        res.append({'MU': row['MU'],
        'Dose': 1 , 
        'Vaccine':txtbd[1] , 
        'Location':txtbd[2] , 
        'Date': txtbd[3]})
    elif( cnt == 1 ):
        res.append({'MU': row['MU'],
        'Dose': '', 
        'Vaccine':'' , 
        'Location':'' , 
        'Date': ''})  
    elif( cnt == 0 ):
        res.append({'MU': row['MU'],
        'Dose': '', 
        'Vaccine':'' , 
        'Location':'' , 
        'Date': ''})        


    return res


lst = []
for index, row in df.iterrows():
    res = parse(row)
    lst.append(res)

flat_list = [item for lst in lst for item in lst]

df1 = pd.DataFrame.from_dict(flat_list)
df1.to_excel('Vaccine_Dataset.xlsx')